In [ ]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image

In [ ]:
BATCH_SIZE = 1
LABELS = sorted([i[:-4] for i in os.listdir('icons')])
NUM_CLASSES = len(LABELS)

In [ ]:
import numpy as np
import collections

class GaussianNoise(object):
    """
    Add gaussian noise to a numpy.ndarray (H x W x C)
    """
    def __init__(self, mean, sigma, random_state=np.random):
        self.sigma = sigma
        self.mean = mean
        self.random_state = random_state

    def __call__(self, image):
        row, col, ch = image.shape
        gauss = self.random_state.normal(self.mean, self.sigma, (row, col, ch))
        gauss = gauss.reshape(row, col, ch)
        image += torch.from_numpy(gauss).float()
        return image

In [ ]:
class IconsDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory
        self.files = os.listdir(directory)[:1000]
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        f = self.files[index]
        # image
        image = Image.open(os.path.join(self.directory, f))
        if self.transform:
            image = self.transform(image)
        
        # labels
        label = f.split('.')[0].split('_')[1]
        label = LABELS.index(label)
        return image, torch.tensor(label, dtype=torch.long)
    
    def input_size(self):
        return 100 * 100 * 3
    
    def __len__(self):
        return len(self.files)


def load_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        GaussianNoise(0.01, 0.001),
    ])
    
    d = IconsDataset('images_single/', LABELS, transform=transform)
    size = len(d)
    train, validate = random_split(d, [int(size * 0.8), int(size * 0.2)])
    
    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

    return d.input_size(), loader, validation_loader

In [ ]:
class ModelConv1(nn.Module):
    def __init__(self, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()
        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=kernel_size)
        conv_layer_output_size = int(100 - (kernel_size - 1))
        self.pooled_pixels = int(conv_layer_output_size / 2)
        self.h1 = nn.Linear(self.pooled_pixels * self.pooled_pixels  * conv_out_channels, linear_size)
        self.h2 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, self.pooled_pixels * self.pooled_pixels * self.conv_out_channels)

        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv2(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        #self.conv3 = nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 2)
        
        x = x - (kernel_size - 1)
        x = int(x / 2)
        
        #x = x - (kernel_size - 1)
        #x = int(x / 2)
        self.pooled_samples = x * x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.max_pool2d(x, 2, 2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.max_pool2d(x, 2, 2)
        
        #x = self.conv3(x)
        #x = F.relu(x)
        #x = F.max_pool2d(x, 2, 2)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
from datetime import datetime

def evalulate(model, validation_loader):
    model.eval()
    loss = 0.0
    correct = 0
    for data, labels in validation_loader:
        labels = labels.cuda()
        predictions_per_class = model(data.cuda())
        _, highest_prediction_class = predictions_per_class.max(1)
        loss += F.nll_loss(predictions_per_class, labels)
        correct += torch.sum(highest_prediction_class == labels)
    return loss/len(validation_loader), correct.item()/len(validation_loader)

def learn(model, loader, validation_loader, epochs=30, learning_rate=0.001):
    torch.cuda.empty_cache()
    optimizer = Adam(params=model.parameters(), lr=learning_rate)

    f = open(f'{datetime.now().isoformat()}.txt', 'w', buffering=1)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        total_correct = 0
        for data, labels in loader:
            labels = labels.cuda()
            predictions_per_class = model(data.cuda())
            highest_prediction, highest_prediction_class = predictions_per_class.max(1)

            # how good are we? compare output with the target classes
            loss = F.nll_loss(predictions_per_class, labels)
            total_loss += loss.item()
            total_correct += torch.sum(highest_prediction_class == labels)

            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        accuracy = total_correct.item()/len(loader)
        train_loss = total_loss/len(loader)
        validation_loss, validation_accuracy = evalulate(model, validation_loader)
        stats = f'Epoch: {epoch}, TL: {train_loss}, VL: {validation_loss.item()}, TA: {accuracy}, VA: {validation_accuracy}'
        print(stats)
        f.write(f'{stats}\n')
        
    return model

In [ ]:
input_size, loader, validation_loader = load_dataset()
# model = ModelConv1(kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
model = ModelConv2(input_size=100, kernel_size=5, conv_out_channels=20, linear_size=500).cuda()
learn(model, loader, validation_loader, epochs=10000, learning_rate=0.0001)